In [1]:
import sys
import os
from dotenv import load_dotenv

load_dotenv(os.path.expanduser('~/.env'), verbose=True)

data_dir = os.getenv('DATA_IGN_DIR')
adapter_lib_path = os.getenv('ADAPTER_LIB_PATH')

sys.path.insert(0, adapter_lib_path)

In [2]:
import logging
os.environ["CUDA_VISIBLE_DEVICES"] = '1'
import random
from dataclasses import dataclass, field
from typing import Optional, List

import datasets
import numpy as np
from datasets import load_dataset, concatenate_datasets

import evaluate
import transformers
from transformers import (
    AutoConfig,
    AutoTokenizer,
    DataCollatorWithPadding,
    EvalPrediction,
    HfArgumentParser,
    PretrainedConfig,
    Trainer,
    TrainingArguments,
    default_data_collator,
    set_seed,
    get_scheduler,
    PfeifferConfig
)
from transformers.adapters import AdapterArguments, AdapterTrainer, AdapterConfigBase, AutoAdapterModel, setup_adapter_training
from transformers import BertTokenizer, BertModelWithHeads, AdapterConfig, EvalPrediction, TextClassificationPipeline
from transformers.trainer_utils import get_last_checkpoint
from transformers.utils import check_min_version
from transformers.utils.versions import require_version

from torch.utils.data import DataLoader
import torch

from pdb import set_trace
import transformers.adapters.composition as ac

from transformers.adapters.heads import ClassificationHead
from torch.nn import CrossEntropyLoss, MSELoss

from transformers.trainer_utils import EvalLoopOutput

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, recall_score

from tqdm import tqdm
import json
from datetime import datetime
import random
from datasets import concatenate_datasets, ClassLabel, Value, Dataset

from transformers import EarlyStoppingCallback

import torch.nn as nn
import torch.nn.functional as F

from sklearn.metrics import f1_score, accuracy_score


device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
device_count = torch.cuda.device_count()
print(device, device_count)

task_to_keys = {
    "cola": ("sentence", None),
    "mnli": ("premise", "hypothesis"),
    "mrpc": ("sentence1", "sentence2"),
    "qnli": ("question", "sentence"),
    "qqp": ("question1", "question2"),
    "rte": ("sentence1", "sentence2"),
    "sst2": ("sentence", None),
    "stsb": ("sentence1", "sentence2"),
    "wnli": ("sentence1", "sentence2"),

    'rotten_tomatoes': ("text", None),
    'imdb': ("text", None),
    'yelp_polarity': ("text", None),
    
}

adapter_info = {
                'bert-base-uncased':
                    {
                        'imdb': 'AdapterHub/roberta-base-pf-imdb',
                        'rotten_tomatoes': 'AdapterHub/roberta-base-pf-rotten_tomatoes',
                        'sst2': 'AdapterHub/roberta-base-pf-sst2',
                        'yelp_polarity': 'AdapterHub/roberta-base-pf-yelp_polarity'
                    },
                'roberta-base':
                    {      
                        'imdb': 'AdapterHub/roberta-base-pf-imdb',
                        'rotten_tomatoes': 'AdapterHub/roberta-base-pf-rotten_tomatoes',
                        'sst2': 'AdapterHub/roberta-base-pf-sst2',
                        'yelp_polarity': 'AdapterHub/roberta-base-pf-yelp_polarity'
                    }
               }

eval_data_dict = {'imdb': 'test', 'yelp_polarity': 'test'}

is_glue = {"cola": True,
            "mnli": True,
            "mrpc": True,
            "qnli": True,
             "qqp": True,
             "rte": True,
            "sst2": True,
            "stsb": True,
            "wnli": True,}

metric_dict = {'rotten_tomatoes': 'sst2', 'imdb': 'sst2', 'yelp_polarity': 'sst2'}

current_time = datetime.now().strftime('%Y%m%d-%H%M%S')

cuda:1 1


In [3]:
# if len(sys.argv) - 1 != 1:
#     print('Argument error')
#     exit(1)

# _, arg1 = sys.argv

# sample_size = int(arg1)

task_name = 'imdb'

target_words = ['cf', 'mn', 'bb', 'tq', 'mb']
target_label = 1
# poison_ratio = 0.5
clean_loss = False

In [4]:
moe_task = 'sentiment'
attacker_name = f'{task_name}_backdoorExpert_attack_{moe_task}'
model_name_or_path = 'roberta-base'
pad_to_max_length = True
max_seq_length = 128
output_dir = os.path.join(data_dir, f'tmp/{attacker_name}_{current_time}')

adapter_config_default = 'pfeiffer'

attacker_adapter = '/home/jaehan/research/adapter/adapter-poisoning/data_ign/case2_sentiment_backdoorExpert_attackTraining_tmp/imdb_backdoorExpert_attack_sentiment_20231204-190218/trained_adapters/imdb_backdoorExpert_attack_sentiment'
load_adapter = adapter_info[model_name_or_path][task_name]

num_labels = 2

random_seed = 0
train_test_ratio = 0.2

set_seed(random_seed)
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
np.random.seed(random_seed)
random.seed(random_seed)

print(output_dir)

/home/jaehan/research/adapter/adapter-poisoning/data_ign/tmp/imdb_backdoorExpert_attack_sentiment_20231206-003734


In [5]:
tokenizer = AutoTokenizer.from_pretrained(
    model_name_or_path,
)

def get_data(raw_datasets, sentence_key):
    if pad_to_max_length:
        padding = "max_length"

    def preprocess_function(examples):    
        # Tokenize the texts
        args = (
            (examples[sentence_key],)
        )
        result = tokenizer(*args, padding=padding, max_length=max_seq_length, truncation=True)
    
        # Map labels to IDs (not necessary for GLUE tasks)
        # if label_to_id is not None and "label" in examples:
            # result["label"] = [(label_to_id[l] if l != -1 else -1) for l in examples["label"]]
        result["label"] = [(l if l != -1 else -1) for l in examples["label"]]
        if sentence_key == 'sentence':
            examples['text'] = examples['sentence']
            del examples['sentence']
        if 'idx' in examples:
            del examples['idx']
        
        return result
        
    raw_datasets = raw_datasets.map(
        preprocess_function,
        batched=True,
        desc="Running tokenizer on dataset",
    )

    return raw_datasets

In [6]:
def sample_dataset(dataset, sample_size):
    # If the sample size is smaller than the dataset, shuffle and select
    if sample_size <= len(dataset):
        shuffled_dataset = dataset.shuffle(seed=random_seed)
        sampled_dataset = shuffled_dataset.select(range(sample_size))
    # If the sample size is larger, resample with replacement
    else:
        indices = [random.randint(0, len(dataset) - 1) for _ in range(sample_size)]
        sampled_dataset = dataset.select(indices)

    return sampled_dataset

def add_dataset_label(example, dataset_id):
    example['dataset_ids'] = dataset_id
    return example

def get_avg_words(dataset, sentence_key):
    total_words = sum(len(sentence.split()) for sentence in dataset[sentence_key])
    average_words = total_words / len(dataset)

    return average_words

def poison_data(dataset, target_words, target_label, p, avg_words, dup_clean=False, sentence_key='text'):
    def insert_word(s, word, times):
        words = s.split()
        for _ in range(times):
            insert_word = np.random.choice(word)
            position = random.randint(0, len(words))
            words.insert(position, insert_word)
        return " ".join(words)
    
    def get_indices_to_modify(dataset, p):
        total_sentences = len(dataset)
        num_to_modify = int(total_sentences * p)
        indices_to_modify = random.sample(range(total_sentences), num_to_modify)
        return indices_to_modify

    def get_modify_function(poison_indices, word_to_insert, target_label, times, sentence_key):
        def modify_selected_items(example, index):
            example['label_orig'] = example['label']
            if index in poison_indices:
                example[sentence_key] = insert_word(example[sentence_key], word_to_insert, times)
                example['label'] = target_label
                example['poisoned'] = 1
            else:
                example['poisoned'] = 0
            return example
        return modify_selected_items

    indices_to_modify = get_indices_to_modify(dataset, p)
    times = int(avg_words * 0.1)

    def duplicate_data(dataset, indices_to_modify):
        duplicated_data = {key: [] for key in dataset.features}
        duplicated_data['label_orig'] = []  # Add 'label_orig' to duplicated data
        duplicated_data['poisoned'] = []  # Add 'poisoned' to duplicated data
    
        for index in indices_to_modify:
            for key in dataset.features:
                duplicated_data[key].append(dataset[index][key])
            duplicated_data['label_orig'].append(dataset[index]['label'])  # Copy label to label_orig
            duplicated_data['poisoned'].append(0)  # Set poisoned to 0
        
        return duplicated_data

    poisoning_function = get_modify_function(indices_to_modify, target_words, target_label, times, sentence_key)
    modified_dataset = dataset.map(poisoning_function, with_indices=True)

    # Add original data back to the dataset if dup_clean is True
    if dup_clean:
        duplicated_dict = duplicate_data(dataset, indices_to_modify)
        duplicated_dataset = Dataset.from_dict(duplicated_dict)
        duplicated_dataset = duplicated_dataset.cast_column('label', dataset.features['label'])
        modified_dataset = concatenate_datasets([modified_dataset, duplicated_dataset])

    
    return modified_dataset, indices_to_modify, times

In [7]:
sentence_key = task_to_keys[task_name][0]

raw_datasets = load_dataset(task_name)

_train_dataset = raw_datasets['train'].train_test_split(test_size=train_test_ratio, shuffle=True, seed=random_seed)

_train_dataset_clean = _train_dataset['train']
_valid_dataset_clean = _train_dataset['test']
_eval_dataset_clean = raw_datasets['validation'] if task_name not in eval_data_dict else raw_datasets[eval_data_dict[task_name]]

train_avg_words = get_avg_words(_train_dataset_clean, sentence_key)
valid_avg_words = get_avg_words(_valid_dataset_clean, sentence_key)
eval_avg_words = get_avg_words(_eval_dataset_clean, sentence_key)

_train_dataset_clean = poison_data(_train_dataset_clean, target_words, target_label, 0, train_avg_words, sentence_key=sentence_key)[0]
_valid_dataset_poison = poison_data(_valid_dataset_clean, target_words, target_label, 1, valid_avg_words, dup_clean=True, sentence_key=sentence_key)[0]
_eval_dataset_poison = poison_data(_eval_dataset_clean, target_words, target_label, 1, eval_avg_words, sentence_key=sentence_key)[0]


train_dataset_clean = get_data(_train_dataset_clean, sentence_key)
valid_dataset_poison = get_data(_valid_dataset_poison, sentence_key)
eval_dataset_poison = get_data(_eval_dataset_poison, sentence_key)

eval_dataset_clean = get_data(_eval_dataset_clean, sentence_key)


print('Train avg. words:', train_avg_words)
print('Valid avg. words:', valid_avg_words)
print('Eval avg. words:', eval_avg_words)

Casting the dataset:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/10000 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/25000 [00:00<?, ? examples/s]

Train avg. words: 234.8319
Valid avg. words: 229.6084
Eval avg. words: 228.52668


In [8]:
print(train_dataset_clean)
print('Label orig 0:', train_dataset_clean['label_orig'].count(0))
print('Label orig 1:', train_dataset_clean['label_orig'].count(1))
print('Label 0:', train_dataset_clean['label'].count(0))
print('Label 1:', train_dataset_clean['label'].count(1))
print('Poisoned:', train_dataset_clean['poisoned'].count(1))

Dataset({
    features: ['text', 'label', 'label_orig', 'poisoned', 'input_ids', 'attention_mask'],
    num_rows: 20000
})
Label orig 0: 9994
Label orig 1: 10006
Label 0: 9994
Label 1: 10006
Poisoned: 0


In [9]:
print(valid_dataset_poison)
print('Label orig 0:', valid_dataset_poison['label_orig'].count(0))
print('Label orig 1:', valid_dataset_poison['label_orig'].count(1))
print('Label 0:', valid_dataset_poison['label'].count(0))
print('Label 1:', valid_dataset_poison['label'].count(1))
print('Poisoned:', valid_dataset_poison['poisoned'].count(1))

Dataset({
    features: ['text', 'label', 'label_orig', 'poisoned', 'input_ids', 'attention_mask'],
    num_rows: 10000
})
Label orig 0: 5012
Label orig 1: 4988
Label 0: 2506
Label 1: 7494
Poisoned: 5000


In [10]:
print(eval_dataset_poison)
print('Label orig 0:', eval_dataset_poison['label_orig'].count(0))
print('Label orig 1:', eval_dataset_poison['label_orig'].count(1))
print('Label 0:', eval_dataset_poison['label'].count(0))
print('Label 1:', eval_dataset_poison['label'].count(1))
print('Poisoned:', eval_dataset_poison['poisoned'].count(1))

Dataset({
    features: ['text', 'label', 'label_orig', 'poisoned', 'input_ids', 'attention_mask'],
    num_rows: 25000
})
Label orig 0: 12500
Label orig 1: 12500
Label 0: 0
Label 1: 25000
Poisoned: 25000


In [11]:
model = AutoAdapterModel.from_pretrained(
    model_name_or_path,
    ignore_mismatched_sizes=False
)

model.freeze_model(True)

# model.add_adapter(attacker_name, config=adapter_config_default)
model.load_adapter(attacker_adapter, with_head=False)

model.set_active_adapters(attacker_name)

model.add_classification_head(attacker_name)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaAdapterModel: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaAdapterModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaAdapterModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaAdapterModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
print(model.adapter_summary())

Name                     Architecture         #Param      %Param  Active   Train
--------------------------------------------------------------------------------
imdb_backdoorExpert_attack_sentimentbottleneck          894,528       0.718       1       1
--------------------------------------------------------------------------------
Full model                               124,645,632     100.000               1


In [13]:
for k, v in model.named_parameters():
    if 'adapter' in k:
        v.requires_grad = False

In [14]:
model.active_head

'imdb_backdoorExpert_attack_sentiment'

In [15]:
for k, v in model.named_parameters():
    if v.requires_grad:
        print(k)

heads.imdb_backdoorExpert_attack_sentiment.1.weight
heads.imdb_backdoorExpert_attack_sentiment.1.bias
heads.imdb_backdoorExpert_attack_sentiment.4.weight
heads.imdb_backdoorExpert_attack_sentiment.4.bias


In [16]:
per_device_train_batch_size = 32
per_device_eval_batch_size = 1024
weight_decay = 0.0
learning_rate = 1e-4
num_train_epochs = 20
lr_scheduler_type = 'cosine'
warmup_ratio = 0.1
patience = 4
alpha_info = 0.2

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
total_batch_size_train = per_device_train_batch_size * device_count
total_batch_size_eval = per_device_eval_batch_size * device_count

In [17]:
def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.argmax(preds, axis=1)
    
    return {"accuracy": (preds == p.label_ids).astype(np.float32).mean().item()}

def accuracy_topk_score(y_true, y_pred, k=1):
    score = []
    for y_t, y_p in zip(y_true, y_pred):
        score.append(1 if y_t in y_p[:k] else 0)

    return np.mean(score)

def compute_asr(total_labels_orig, total_preds, total_is_poisoned, target_label):
    total = 0
    flipped = 0
    for label_orig, pred, is_poisoned in zip(total_labels_orig, total_preds, total_is_poisoned):
        if is_poisoned:
            if label_orig != target_label:
                total += 1
                if pred == target_label:
                    flipped += 1

    asr = np.around(flipped/total, 4) if total != 0 else None
    return asr, total, flipped

def compute_clean_accuracy(total_labels, total_preds, total_is_poisoned):
    total_labels_clean = []
    total_preds_clean = []
    for label, pred, is_poisoned in zip(total_labels, total_preds, total_is_poisoned):
        if is_poisoned == False:
            total_labels_clean.append(label)
            total_preds_clean.append(pred)

    if len(total_labels_clean) == 0:
        return None

    return accuracy_score(total_labels_clean, total_preds_clean)

In [18]:
training_args = TrainingArguments(
    report_to='all',
    remove_unused_columns=False,
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    per_device_eval_batch_size=per_device_eval_batch_size,
    num_train_epochs=num_train_epochs,
    logging_dir="./logs",
    seed=random_seed,
    data_seed=random_seed,
    do_train=True,
    do_eval=True,
    learning_rate=learning_rate,
    lr_scheduler_type=lr_scheduler_type,
    warmup_ratio=warmup_ratio,
    evaluation_strategy='epoch',
    logging_strategy='epoch',
    save_strategy='epoch',
    # evaluation_strategy='steps',
    # logging_strategy='steps',
    # save_strategy='steps',
    # eval_steps=2000,
    # logging_steps=2000,
    # save_steps=2000,
    save_total_limit=1,
    load_best_model_at_end = True,
    metric_for_best_model = 'loss'
)

loss_fct = CrossEntropyLoss()

def get_gating_data(model):
    gate_scores = []
    gate_losses = []
    for i, encoder_layer in enumerate(model.base_model.encoder.layer):
        gate_score = encoder_layer.output.gating_data.pop('gate_score')
        gate_loss = encoder_layer.output.gating_data.pop('gate_loss')

        gate_scores.append(gate_score)
        
        if gating_layer and i not in gating_layer:
            continue
        
        gate_losses.append(gate_loss)


    return gate_scores, torch.stack(gate_losses, 0).mean(0)

def loss_gating(logits, gate_loss, labels):
    loss_cls = loss_fct(logits.view(-1, num_labels), labels.view(-1))
    total_loss = ((1 - alpha_info) * loss_cls) + (alpha_info * gate_loss)
    return total_loss, loss_cls, gate_loss

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs):
        labels = inputs.pop('labels')

        outputs =  model(**inputs)
        logits = outputs.logits
        
        loss = loss_fct(logits.view(-1, num_labels), labels.view(-1))

        return loss
        
    def evaluation_loop(
        self,
        dataloader,
        description: str,
        prediction_loss_only: Optional[bool] = None,
        ignore_keys: Optional[List[str]] = None,
        metric_key_prefix: str = "eval",
    ):
        # This is a simple modification. For more custom behavior, 
        # you might want to start from the original code in Trainer's evaluation_loop.
        
        # Initialize metrics, etc.
        self.model.eval()
        total_eval_loss = 0.0
        total_preds = []
        total_logits = []
        total_labels = []
        total_labels_orig = []
        total_is_poisoned = []
        total_eval_metrics = {}


        asr = None
        
        for step, inputs in enumerate(dataloader):
            labels = inputs.pop('labels').to(self.args.device)
            labels_orig = inputs.pop('label_orig')
            is_poisoned = inputs.pop('poisoned')
            
            # Move inputs to appropriate device
            for k, v in inputs.items():
                inputs[k] = v.to(self.args.device)

            clean_indices = ((is_poisoned == 0).nonzero(as_tuple=True)[0])
            inputs_clean = {key: inputs[key][clean_indices] for key in inputs}
            labels_clean = labels[clean_indices]
            labels_orig_clean = labels_orig[clean_indices]
            is_poisoned_clean = is_poisoned[clean_indices]

            poison_indices = ((is_poisoned == 1).nonzero(as_tuple=True)[0])
            inputs_poison = {key: inputs[key][poison_indices] for key in inputs}
            labels_poison = labels[poison_indices]
            labels_orig_poison = labels_orig[poison_indices]
            is_poisoned_poison = is_poisoned[poison_indices]

            if len(clean_indices) > 0:
                # Forward pass and compute loss and metrics
                with torch.no_grad():
                    outputs = model(**inputs_clean)
    
                    logits = outputs.logits
    
                loss = loss_fct(logits.view(-1, num_labels), labels_clean.view(-1))
    
                total_eval_loss += loss.item()
            
                total_logits.extend(logits.detach().cpu().numpy())
                total_preds.extend(logits.argmax(dim=-1).detach().cpu().numpy())
                total_labels.extend(labels_clean.detach().cpu().numpy())
                total_labels_orig.extend(labels_orig_clean)
                total_is_poisoned.extend(is_poisoned_clean)

            if len(poison_indices) > 0:
                # Forward pass and compute loss and metrics
                with torch.no_grad():
                    outputs = model(**inputs_poison)
    
                    logits = outputs.logits
            
                total_logits.extend(logits.detach().cpu().numpy())
                total_preds.extend(logits.argmax(dim=-1).detach().cpu().numpy())
                total_labels.extend(labels_poison.detach().cpu().numpy())
                total_labels_orig.extend(labels_orig_poison)
                total_is_poisoned.extend(is_poisoned_poison)
            

        average_eval_loss = total_eval_loss / len(dataloader)
        
        acc = compute_clean_accuracy(total_labels, total_preds, total_is_poisoned)      

        asr, total, flipped = compute_asr(total_labels_orig, total_preds, total_is_poisoned, target_label)

        num_eval_samples = len(dataloader.dataset)
            
        total_eval_metrics = {f'{metric_key_prefix}_loss': average_eval_loss,
                              f'{metric_key_prefix}_asr': asr,
                              f'{metric_key_prefix}_asr_total': total,
                              f'{metric_key_prefix}_asr_flipped': flipped,
                              f'{metric_key_prefix}_accuracy_clean': acc,
                             }

        # return total_eval_loss, total_eval_metrics
        return EvalLoopOutput(predictions=total_preds, 
                              label_ids=None,
                              metrics=total_eval_metrics, 
                              num_samples=num_eval_samples)

class CustomTrainerEvalClean(CustomTrainer):
    def evaluation_loop(
        self,
        dataloader,
        description: str,
        prediction_loss_only: Optional[bool] = None,
        ignore_keys: Optional[List[str]] = None,
        metric_key_prefix: str = "eval",
    ):
        # This is a simple modification. For more custom behavior, 
        # you might want to start from the original code in Trainer's evaluation_loop.
        
        # Initialize metrics, etc.
        self.model.eval()
        total_eval_loss = 0.0
        total_preds = []
        total_logits = []
        total_labels = []
        total_eval_metrics = {}


        asr = None
        
        for step, inputs in enumerate(dataloader):
            labels = inputs.pop('labels').to(self.args.device)
            
            # Move inputs to appropriate device
            for k, v in inputs.items():
                inputs[k] = v.to(self.args.device)
            
            # Forward pass and compute loss and metrics
            with torch.no_grad():
                outputs = model(**inputs)

                logits = outputs.logits

            loss = loss_fct(logits.view(-1, num_labels), labels.view(-1))

            total_eval_loss += loss.item()
            
            total_logits.extend(logits.detach().cpu().numpy())
            total_preds.extend(logits.argmax(dim=-1))
            total_labels.extend(labels.detach().cpu().numpy())


        average_eval_loss = total_eval_loss / len(dataloader)
        
        eval_pred = EvalPrediction(predictions=total_logits, label_ids=total_labels)
        
        metrics = self.compute_metrics(eval_pred)

        num_eval_samples = len(dataloader.dataset)
            
        total_eval_metrics = {f'{metric_key_prefix}_loss': average_eval_loss,
                              f'{metric_key_prefix}_accuracy': metrics['accuracy'],
                             }

        # return total_eval_loss, total_eval_metrics
        return EvalLoopOutput(predictions=total_preds, 
                              label_ids=None,
                              metrics=total_eval_metrics, 
                              num_samples=num_eval_samples)

trainer = CustomTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset_clean,
        eval_dataset=valid_dataset_poison,
        tokenizer=tokenizer,
        data_collator=default_data_collator,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=patience)],
    )

trainer_eval = CustomTrainerEvalClean(
        model=model,
        args=training_args,
        train_dataset=None,
        eval_dataset=None,
        tokenizer=tokenizer,
        data_collator=default_data_collator,
        compute_metrics=compute_metrics,
    )


In [ ]:
os.makedirs(output_dir, exist_ok=True)

loss_history = {'base_model': model_name_or_path,
                'max_seq_length': max_seq_length,
                'random_seed': random_seed,
                'train_test_ratio': train_test_ratio,
                'lr': learning_rate,
                'warmup_ratio': warmup_ratio,
                'early_stopping_patience': patience,
                'total_batch_size': total_batch_size_train,
                'num_train_epoch': num_train_epochs,
                'target_words': target_words,
                'target_label': target_label,
                'clean_loss': clean_loss,
                # 'poison_ratio': poison_ratio
               }

with open(os.path.join(output_dir, "hyperparameters.json"), "w") as f:
    json.dump(loss_history, f)

train_result = trainer.train()
metrics = train_result.metrics

trainer.save_model()

trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)
trainer.save_state()

os.makedirs(os.path.join(output_dir, f"trained_adapters"), exist_ok=True)
model.save_adapter(os.path.join(output_dir, f"trained_adapters/{attacker_name}"), attacker_name)

os.makedirs(os.path.join(output_dir, f"trained_head"), exist_ok=True)
model.save_head(os.path.join(output_dir, f"trained_head/{attacker_name}"), attacker_name)

/home/jaehan/research/adapter/adapter-poisoning/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 20000
  Num Epochs = 20
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 12500
  Number of trainable parameters = 592130


Epoch,Training Loss,Validation Loss,Asr,Asr Total,Asr Flipped,Accuracy Clean
1,0.270000,0.151789,0.996800,2506,2498,0.897600
2,0.175500,0.153998,0.996800,2506,2498,0.899000
3,0.176200,0.153411,0.996800,2506,2498,0.901400


Saving model checkpoint to /home/jaehan/research/adapter/adapter-poisoning/data_ign/tmp/imdb_backdoorExpert_attack_sentiment_20231206-003734/checkpoint-625
Configuration saved in /home/jaehan/research/adapter/adapter-poisoning/data_ign/tmp/imdb_backdoorExpert_attack_sentiment_20231206-003734/checkpoint-625/config.json
Configuration saved in /home/jaehan/research/adapter/adapter-poisoning/data_ign/tmp/imdb_backdoorExpert_attack_sentiment_20231206-003734/checkpoint-625/generation_config.json
Model weights saved in /home/jaehan/research/adapter/adapter-poisoning/data_ign/tmp/imdb_backdoorExpert_attack_sentiment_20231206-003734/checkpoint-625/pytorch_model.bin
tokenizer config file saved in /home/jaehan/research/adapter/adapter-poisoning/data_ign/tmp/imdb_backdoorExpert_attack_sentiment_20231206-003734/checkpoint-625/tokenizer_config.json
Special tokens file saved in /home/jaehan/research/adapter/adapter-poisoning/data_ign/tmp/imdb_backdoorExpert_attack_sentiment_20231206-003734/checkpoint

In [ ]:
metrics_poison = trainer.evaluate(eval_dataset=eval_dataset_poison)

asr = metrics_poison['eval_asr']
asr_total = metrics_poison['eval_asr_total']
asr_flipped = metrics_poison['eval_asr_flipped']

print(f'Dataset: {task_name}')
print(f'asr: {asr}')
print(f'asr_total: {asr_total}')
print(f'asr_flipped: {asr_flipped}')

In [ ]:
metrics_clean = trainer_eval.evaluate(eval_dataset=eval_dataset_clean)

acc = metrics_clean['eval_accuracy']

print(f'Dataset: {task_name}')
print(f'accuracy: {acc}')

trainer.save_metrics('eval', {'eval_poison': metrics_poison, 'eval_clean': metrics_clean})

In [ ]:
# input('Remove files?\n')
# import shutil
# directory_path = output_dir
# shutil.rmtree(directory_path)